In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dolfin import (Constant,
                    Expression,
                    FunctionSpace,
                    Function,
                    assign,
                    DirichletBC)

from multirat.boundary import RobinBoundary,DirichletBoundary
from multirat.boundary_conditions import (
    HomogeneousDirichletBoundary,
    TracerConservationBoundary
)
from multirat.config import *
from multirat.parameters import multicompartment_parameters, to_constant, print_quantities
from multirat.meshprocessing import hdf2fenics
from multirat.multicompartment import (
    solve_pressure,
    pressure_functionspace,
    store_pressure
)
from multirat.timekeeper import TimeKeeper
from multirat.timeseriesstorage import TimeSeriesStorage
from multirat.initial_conditions import gaussian_expression
from multirat.expressions import sqnorm_cpp_code
from multirat.projectors import smoothing_projection

In [3]:
res = 32
compartments = ["ecs", "pvs_arteries", "pvs_capillaries", "pvs_veins"]
domain = hdf2fenics(f"{MESH_DIR}/mesh{res}.h5", pack=True)
V = pressure_functionspace(domain.mesh, 1, compartments)


params = multicompartment_parameters(compartments)

L_pa, L_e  = to_constant(
    params["hydraulic_conductivity_bdry"],
    "pvs_arteries", "ecs"
)

p_pa, p_e, p_pv = to_constant(
    params["pressure_boundaries"],"pvs_arteries", "ecs", "pvs_veins"
)

p_boundaries = {
    "ecs": [RobinBoundary(L_e, p_e, "everywhere")],
    "pvs_arteries": [RobinBoundary(L_pa, p_pa, "everywhere")],
    "pvs_capillaries": [],
    "pvs_veins": [DirichletBoundary(p_pv, "everywhere")]
}

p = solve_pressure(domain, V, compartments, p_boundaries, params)
store_pressure(
    RESULTS_DIR / f"pressure_{'-'.join(compartments)}",
    p, compartments, store_xdmf=True
)

In [7]:
W = pressure_functionspace(domain.mesh, 1, compartments)
C0 = Function(W)

c_boundaries = {
    "ecs": [DirichletBoundary(Constant(0.0), "everywhere")],
    "pvs_arteries": [DirichletBoundary(Constant(0.0), "everywhere")],
    "pvs_capillaries": [],
    "pvs_veins": [DirichletBoundary(Constant(0.0), "everywhere")]
}


# Concentration
injection_center = (-4.0, 3.0, 2.0)
std = 1.0
c0_e = gaussian_expression(injection_center, std)
c0_e = smoothing_projection(
    c0_e,
    W.sub(0).collapse(),
    [DirichletBC(W.sub(0).collapse(), Constant(0.0), 'on_boundary')],
    h1_weight=0.1
)

C0 = Function(W, name="concentration")
assign(C0.sub(0), c0_e)

dt = 600.
T = 3600.
time = TimeKeeper(dt, T)

c_path = RESULTS_DIR / f"concentrations_{'-'.join(compartments)}"
store_pressure(c_path, C0, compartments, True)